# Web Scraping for Indeed.com & Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary classifier.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use a random forest regressor, as well as another classifier of your choice; either logistic regression, SVM, or KNN. 

- **Question**: Why would we want this to be a classification problem?
- **Answer**: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using requests) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)
The URL here has many query parameters
- q for the job search
- This is followed by "+20,000" to return results with salaries (or expected salaries >$20,000)
- l for a location
- start for what result number to start on

In [25]:
#Using a random forest regressor, with one other classifier. 

In [26]:
url = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"


In [27]:
import requests
import bs4
from bs4 import BeautifulSoup
import urllib
html = urllib.urlopen(url).read()

In [28]:
html

'<!DOCTYPE html>\n<html lang="en">\n<head>\n<meta http-equiv="content-type" content="text/html;charset=UTF-8">\n<!-- pll --><script type="text/javascript" src="/s/7331891/en_US.js"></script>\n<link href="/s/ff82884/jobsearch_all.css" rel="stylesheet" type="text/css">\n<link rel="alternate" type="application/rss+xml" title="Data Scientist $20,000 Jobs, Employment in New York, NY" href="http://rss.indeed.com/rss?q=data+scientist+%2420%2C000&l=New+York">\n<link rel="alternate" media="handheld" href="/m/jobs?q=data+scientist+%2420%2C000&l=New+York">\n<script type="text/javascript">\n    \n    window[\'closureReadyCallbacks\'] = [];\n\n    function call_when_jsall_loaded(cb) {\n        if (window[\'closureReady\']) {\n            cb();\n        } else {\n            window[\'closureReadyCallbacks\'].push(cb);\n        }\n    }\n</script>\n\n<script type="text/javascript" src="/s/d6a62e8/jobsearch-all-compiled.js"></script>\n            <script type="text/javascript">\nvar pingUrlsForGA = []

In [29]:
b = BeautifulSoup(html, 'html.parser', from_encoding="utf-8")

In [30]:
#http://stackoverflow.com/questions/9907492/how-to-get-firefox-working-with-selenium-webdriver-on-mac-osx

In [31]:
## YOUR CODE HERE
b.find_all('span', {'class','summary'}) 

[<span class="summary">Entrepreneurial market intelligence firm serving the dynamic mobile telecommunications industry is seeking to hire a Research <b>Data</b> Analyst....</span>,
 <span class="summary">Extracts <b>data</b> for publications, or provides <b>data</b> collection from outside physicians offices. Reviews <b>data</b> with supervisor and provides reports to the <b>data</b> and...</span>,
 <span class="summary">We leverage our petabytes-sized <b>data</b> clusters to derive industry leading analytics and provide unique insights for our customers....</span>,
 <span class="summary" itemprop="description">\nAssociate Research <b>Scientist</b>*. The <b>Data</b> <b>Scientist</b>/Engineer will be part of an application development group of clinicians, <b>scientists</b> and engineers....</span>,
 <span class="summary" itemprop="description">\nThe <b>Data</b> Science team builds internal and external <b>data</b> products to service our merchants and our company alike....</span>,
 <spa

In [32]:
#List of summaries for New York 20,000.
for entry in b.find_all('span', {'class','summary'}):
    print entry.text

Entrepreneurial market intelligence firm serving the dynamic mobile telecommunications industry is seeking to hire a Research Data Analyst....
Extracts data for publications, or provides data collection from outside physicians offices. Reviews data with supervisor and provides reports to the data and...
We leverage our petabytes-sized data clusters to derive industry leading analytics and provide unique insights for our customers....

Associate Research Scientist*. The Data Scientist/Engineer will be part of an application development group of clinicians, scientists and engineers....

The Data Science team builds internal and external data products to service our merchants and our company alike....

Data Scientist - Research, Insights & Segmentation. As Data Scientist, our mission is to turn terabytes of data into insights and get a deep understanding of...

Lead Data Scientist, Digital Intelligence. Proprietary data platforms. They are hands-on, without needing an army of engineers or

Let's look at one result more closely. A single result looks like
```JSON
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&campaignid=serp-linkcompanyname&fromjk=2480d203f7e97210&jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a nobr element inside of a td element with class='snip.
- The title of a job is in a link with class set to jobtitle and a data-tn-element="jobTitle.
- The location is set in a span with class='location'.
- The company is set in a span with class='company'.

## Write 4 functions to extract each item: location, company, job, and salary.¶
Example
```python
def extract_location_from_result(result):
    return result.find ...
```

##### - Make sure these functions are robust and can handle cases where the data/field may not be available.
>- Remember to check if a field is empty or None for attempting to call methods on it
>- Remember to use try/except if you anticipate errors.

- **Test** the functions on the results above and simple examples

In [33]:
def extract_job_from_result(result): 
    url = result 
    html = urllib.urlopen(url).read()
    b = BeautifulSoup(html, 'html.parser', from_encoding="utf-8")
    for entry in b.find_all('h2', {'class':'jobtitle'}):
        entry.text

In [34]:
extract_job_from_result('http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10')

In [35]:
def extract_location_from_result(result): 
    url = result 
    html = urllib.urlopen(url).read()
    b = BeautifulSoup(html, 'html.parser', from_encoding="utf-8")
    for entry in b.find_all('span', {'class':'location'}):
        entry.text

In [36]:
extract_location_from_result('https://www.indeed.com/jobs?q=data+scientist+$20,000&l=New+York&start=10')

In [37]:
def extract_company_from_result(result):
    url = result 
    html = urllib.urlopen(url).read()
    b = BeautifulSoup(html, 'html.parser', from_encoding="utf-8")
    for entry in b.find_all('span', {'class':'company'}):
        entry.text

In [38]:
extract_company_from_result('https://www.indeed.com/jobs?q=data+scientist+$20,000&l=New+York&start=10')

In [39]:
#The salary is available in a nobr element inside of a td element with class='snip'.
def extract_salary_from_result(result):
    url = result 
    html = urllib.urlopen(url).read()
    b = BeautifulSoup(html, 'html.parser', from_encoding="utf-8")
    for entry in b.find_all('td', {'class':'snip'}):
        try:
            entry.find('nobr').renderContents()
        except:
            'NONE LISTED'

In [40]:
extract_salary_from_result('http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10')

Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.
- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the l=New+York and the start=10. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).
##### Complete the following code to collect results from multiple cities and starting points.
- Enter your city below to add it to the search
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

In [41]:
YOUR_CITY = 'Boston'

In [42]:
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 10000 # Set this to a high-value (5000) to generate more results. 
# Crawling more results, will also take much longer. First test your code on a small number of results and then expand.

results = []

for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 
    'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
    'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', YOUR_CITY]):
    for start in range(0, max_results_per_city, 10):
        # Grab the results from the request (as above)
        url = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=" + city +"&start="+ str(start)
        # Append to the full set of results
        results.append(url)
        pass

In [43]:
results

['http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=Houston&start=0',
 'http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=Houston&start=10',
 'http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=Houston&start=20',
 'http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=Houston&start=30',
 'http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=Houston&start=40',
 'http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=Houston&start=50',
 'http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=Houston&start=60',
 'http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=Houston&start=70',
 'http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=Houston&start=80',
 'http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=Houston&start=90',
 'http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=Houston&start=100',
 'http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=Houston&start=110',
 'http://www.indeed.com/jobs?q=data+scientist+%2420

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [44]:
import pandas as pd
job_details = pd.DataFrame(columns=['location','title','company', 'salary'])

In [48]:
#Take the overall entry and excract info from there first. 
for result in results:
    url = result 
    html = urllib.urlopen(url).read()
    b = BeautifulSoup(html, 'html.parser', from_encoding="utf-8")
    for entry in b.find_all('span', {'class':'location'}):
        location = entry.text
    for entry in b.find_all('h2', {'class':'jobtitle'}):
        title = entry.text
    for entry in b.find_all('span', {'class':'company'}):
        company = entry.text
    for entry in b.find_all('td', {'class':'snip'}):
        try:
            salary = entry.find('nobr').renderContents()
        except:
            salary = 'NONE LISTED'
    job_details.loc[len(job_details)]=[location, title, company, salary]    

In [49]:
job_details

,location,title,company,salary
0,"Houston, TX 77058",\nDATA SCIENTIST\n,\n Cyberonics,NONE LISTED
1,United States,\nData Scientist\n,\n\n Amazon Corporate LLC,NONE LISTED
2,"Sugar Land, TX",\nUltimate Hoops Statistician\n,\n\n\n Lifetime Fitness\n,NONE LISTED
3,"Houston, TX",\nData Scientist - TX - 5298\n,"\n\n DB Healthcare, Inc.\n",NONE LISTED
4,"Houston, TX 77070",\nData Scientist\n,"\n\n\n Noble Energy, Inc.\n",NONE LISTED
5,"Houston, TX",\nAir Monitoring Specialist -PM 2.5\n,\n\n\n Alion Science and Technology\n,NONE LISTED
6,"Houston, TX",\nPostdoctoral Fellow/Research Scientist Posit...,\n\n\n University of Houston\n,NONE LISTED
7,"Houston, TX",\nClinical Research Financial Analyst\n,\n\n\n UT Health Science Center at Hous...,NONE LISTED
8,"Houston, TX 77042 (West Houston area)",\nHead of Engineering\n,\n\n\n Harris Corporation\n,NONE LISTED
9,"Houston, TX",\nResearch Scientist – Advanced Materials\n,\n\n C-Crete Technologies\n,NONE LISTED


Lastly, we need to clean up salary data. 

1. Only a small number of the scraped results have salary information - only these will be used for modeling.
1. Some of the salaries are not yearly but hourly or weekly, these will not be useful to us for now
1. Some of the entries may be duplicated
1. The salaries are given as text and usually with ranges.

#### Find the entries with annual salary entries, by filtering the entries without salaries or salaries that are not yearly (filter those that refer to hour or week). Also, remove duplicate entries

In [51]:
job_details = job_details[job_details.salary != 'NONE LISTED']

In [52]:
job_details

,index,location,title,company,salary
1959,1959,"Boston, MA",\nSAS Statistical Data Analyst-Healthcare Mark...,\n\n ALC Staffing Associates\n,"$70,000 - $75,000 a year"
1968,1968,"Boston, MA",\nSAS Program / Statistical Analyst-Data Mining\n,\n\n Alexander Bec\n,"$75,000 a year"
1974,1974,"Andover, MA",\nStatistical Analyst\n,\n\n\n GCM\n,$28 an hour
1998,1998,"Boston, MA 02108 (Back Bay-Beacon Hill area)",\nData Analyst\n,\n\n Infotek Consulting Services Inc.\n,"$100,000 - $110,000 a year"
2007,2007,"Framingham, MA",\nStatistician II\n,\n\n IconsultUS\n,$43 an hour
2528,2528,"Cambridge, MA",\nFront-End Developer - JavaScript/HTML5 for E...,\n\n The Bivium Group\n,"$75,000 - $130,000 a year"
2529,2529,"Cambridge, MA",\nFront-End Developer - JavaScript/HTML5 for E...,\n\n The Bivium Group\n,"$75,000 - $130,000 a year"
2537,2537,"Houston, TX",\nQuantitative Risk Analyst\n,\n\n SearchBankingJobs\n,"$150,000 - $205,000 a year"
3550,3550,"Cambridge, MA 02139 (Area IV area)",\nDatabase Engineer/ SPARK Engineer\n,\n\n\n Aequor Technologies\n,$55 an hour
3554,3554,"Lexington, MA 02420",\nStatistical Programmer\n,\n\n BioPier Inc.\n,"$60,000 - $120,000 a year"


In [53]:
job_details = job_details.reset_index()

In [56]:
job_details = job_details.drop('index', 1)

In [59]:
job_details = job_details.drop('level_0', 1)

In [60]:
job_details

,location,title,company,salary
0,"Boston, MA",\nSAS Statistical Data Analyst-Healthcare Mark...,\n\n ALC Staffing Associates\n,"$70,000 - $75,000 a year"
1,"Boston, MA",\nSAS Program / Statistical Analyst-Data Mining\n,\n\n Alexander Bec\n,"$75,000 a year"
2,"Andover, MA",\nStatistical Analyst\n,\n\n\n GCM\n,$28 an hour
3,"Boston, MA 02108 (Back Bay-Beacon Hill area)",\nData Analyst\n,\n\n Infotek Consulting Services Inc.\n,"$100,000 - $110,000 a year"
4,"Framingham, MA",\nStatistician II\n,\n\n IconsultUS\n,$43 an hour
5,"Cambridge, MA",\nFront-End Developer - JavaScript/HTML5 for E...,\n\n The Bivium Group\n,"$75,000 - $130,000 a year"
6,"Cambridge, MA",\nFront-End Developer - JavaScript/HTML5 for E...,\n\n The Bivium Group\n,"$75,000 - $130,000 a year"
7,"Houston, TX",\nQuantitative Risk Analyst\n,\n\n SearchBankingJobs\n,"$150,000 - $205,000 a year"
8,"Cambridge, MA 02139 (Area IV area)",\nDatabase Engineer/ SPARK Engineer\n,\n\n\n Aequor Technologies\n,$55 an hour
9,"Lexington, MA 02420",\nStatistical Programmer\n,\n\n BioPier Inc.\n,"$60,000 - $120,000 a year"


In [61]:
## YOUR CODE HERE
job_details = job_details[job_details.salary.str.contains("a month") == False]
job_details = job_details[job_details.salary.str.contains("an hour") == False]
job_details = job_details[job_details.salary.str.contains("a week") == False]
job_details = job_details[job_details.salary.str.contains("a day") == False]

In [147]:
job_details

,location,title,company,salary
0,"Boston, MA",SAS Statistical Data Analyst-Healthcare Marketing,ALC Staffing Associates,70000 75000
1,"Boston, MA",SAS Program / Statistical Analyst-Data Mining,Alexander Bec,75000
3,"Boston, MA 02108 (Back Bay-Beacon Hill area)",Data Analyst,Infotek Consulting Services Inc.,100000 110000
5,"Cambridge, MA",Front-End Developer - JavaScript/HTML5 for EdT...,The Bivium Group,75000 130000
6,"Cambridge, MA",Front-End Developer - JavaScript/HTML5 for EdT...,The Bivium Group,75000 130000
7,"Houston, TX",Quantitative Risk Analyst,SearchBankingJobs,150000 205000
9,"Lexington, MA 02420",Statistical Programmer,BioPier Inc.,60000 120000
11,"Boston, MA 02116 (South End area)",Back End Java Developer-Machine Learning,Jobspring Partners,130000
12,"Boston, MA",Spark/Hadoop/Scala at Top Growth Stage Startup,Workbridge Associates,100000 165000
13,"Boston, MA",Principal Software Engineer Lead Software Engi...,The Bivium Group,120000 165000


#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [63]:
## YOUR CODE HERE
job_details['salary'] = (job_details['salary'].replace( '[\a year,)]','', regex=True))
job_details['salary'] = (job_details['salary'].replace( '[\$,)]','', regex=True)) 
job_details['company'] = (job_details['company'].replace( '[\\n,)]','', regex=True))
job_details['company'] = (job_details['company'].replace( '[\\n\n,)]','', regex=True)) 
job_details['title'] = (job_details['title'].replace( '[\\n,)]','', regex=True))

In [132]:
all_jobs

,location,title,company,salary
0,"Boston, MA",SAS Statistical Data Analyst-Healthcare Marketing,ALC Staffing Associates,70000 75000
1,"Boston, MA",SAS Program / Statistical Analyst-Data Mining,Alexander Bec,75000
3,"Boston, MA 02108 (Back Bay-Beacon Hill area)",Data Analyst,Infotek Consulting Services Inc.,100000 110000
5,"Cambridge, MA",Front-End Developer - JavaScript/HTML5 for EdT...,The Bivium Group,75000 130000
6,"Cambridge, MA",Front-End Developer - JavaScript/HTML5 for EdT...,The Bivium Group,75000 130000
7,"Houston, TX",Quantitative Risk Analyst,SearchBankingJobs,150000 205000
9,"Lexington, MA 02420",Statistical Programmer,BioPier Inc.,60000 120000
11,"Boston, MA 02116 (South End area)",Back End Java Developer-Machine Learning,Jobspring Partners,130000
12,"Boston, MA",Spark/Hadoop/Scala at Top Growth Stage Startup,Workbridge Associates,100000 165000
13,"Boston, MA",Principal Software Engineer Lead Software Engi...,The Bivium Group,120000 165000


In [388]:
#Checkpoint.
job_details = all_jobs

In [389]:
job_details

,location,title,company,salary
0,"Boston, MA",SAS Statistical Data Analyst-Healthcare Marketing,ALC Staffing Associates,70000 75000
1,"Boston, MA",SAS Program / Statistical Analyst-Data Mining,Alexander Bec,75000
3,"Boston, MA 02108 (Back Bay-Beacon Hill area)",Data Analyst,Infotek Consulting Services Inc.,100000 110000
5,"Cambridge, MA",Front-End Developer - JavaScript/HTML5 for EdT...,The Bivium Group,75000 130000
6,"Cambridge, MA",Front-End Developer - JavaScript/HTML5 for EdT...,The Bivium Group,75000 130000
7,"Houston, TX",Quantitative Risk Analyst,SearchBankingJobs,150000 205000
9,"Lexington, MA 02420",Statistical Programmer,BioPier Inc.,60000 120000
11,"Boston, MA 02116 (South End area)",Back End Java Developer-Machine Learning,Jobspring Partners,130000
12,"Boston, MA",Spark/Hadoop/Scala at Top Growth Stage Startup,Workbridge Associates,100000 165000
13,"Boston, MA",Principal Software Engineer Lead Software Engi...,The Bivium Group,120000 165000


In [390]:
job_details_2 = job_details.drop_duplicates()
#533 results. Left with 34 in total. 

In [391]:
job_details_2

,location,title,company,salary
0,"Boston, MA",SAS Statistical Data Analyst-Healthcare Marketing,ALC Staffing Associates,70000 75000
1,"Boston, MA",SAS Program / Statistical Analyst-Data Mining,Alexander Bec,75000
3,"Boston, MA 02108 (Back Bay-Beacon Hill area)",Data Analyst,Infotek Consulting Services Inc.,100000 110000
5,"Cambridge, MA",Front-End Developer - JavaScript/HTML5 for EdT...,The Bivium Group,75000 130000
7,"Houston, TX",Quantitative Risk Analyst,SearchBankingJobs,150000 205000
9,"Lexington, MA 02420",Statistical Programmer,BioPier Inc.,60000 120000
11,"Boston, MA 02116 (South End area)",Back End Java Developer-Machine Learning,Jobspring Partners,130000
12,"Boston, MA",Spark/Hadoop/Scala at Top Growth Stage Startup,Workbridge Associates,100000 165000
13,"Boston, MA",Principal Software Engineer Lead Software Engi...,The Bivium Group,120000 165000
14,"Boston, MA",Clinical Laboratory Surveyor,Commonwealth of Massachusetts,60096 87045


In [392]:
#Need to convert the ranges. 

In [393]:
job_details_2['salary'] = (job_details_2['salary'].replace( '[\-,)]',' ', regex=True))  

/Applications/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [394]:
job_details_2.reset_index()

,index,location,title,company,salary
0,0,"Boston, MA",SAS Statistical Data Analyst-Healthcare Marketing,ALC Staffing Associates,70000 75000
1,1,"Boston, MA",SAS Program / Statistical Analyst-Data Mining,Alexander Bec,75000
2,3,"Boston, MA 02108 (Back Bay-Beacon Hill area)",Data Analyst,Infotek Consulting Services Inc.,100000 110000
3,5,"Cambridge, MA",Front-End Developer - JavaScript/HTML5 for EdT...,The Bivium Group,75000 130000
4,7,"Houston, TX",Quantitative Risk Analyst,SearchBankingJobs,150000 205000
5,9,"Lexington, MA 02420",Statistical Programmer,BioPier Inc.,60000 120000
6,11,"Boston, MA 02116 (South End area)",Back End Java Developer-Machine Learning,Jobspring Partners,130000
7,12,"Boston, MA",Spark/Hadoop/Scala at Top Growth Stage Startup,Workbridge Associates,100000 165000
8,13,"Boston, MA",Principal Software Engineer Lead Software Engi...,The Bivium Group,120000 165000
9,14,"Boston, MA",Clinical Laboratory Surveyor,Commonwealth of Massachusetts,60096 87045


In [395]:
salaries = job_details_2.salary.str.split(' ', expand=True)

In [396]:
salaries 

,0,1
0,70000,75000
1,75000,None
3,100000,110000
5,75000,130000
7,150000,205000
9,60000,120000
11,130000,None
12,100000,165000
13,120000,165000
14,60096,87045


In [397]:
salaries = salaries.astype(float)

In [398]:
salaries.dtypes

0    float64
1    float64
dtype: object

In [399]:
salaries = salaries.rename(columns = {0:'salary_1', 1:'salary_2'})

In [400]:
salaries

,salary_1,salary_2
0,70000.0,75000.0
1,75000.0,NaN
3,100000.0,110000.0
5,75000.0,130000.0
7,150000.0,205000.0
9,60000.0,120000.0
11,130000.0,NaN
12,100000.0,165000.0
13,120000.0,165000.0
14,60096.0,87045.0


In [401]:
salaries.salary_2 = salaries.salary_2.fillna(salaries.salary_1)

In [402]:
salaries

,salary_1,salary_2
0,70000.0,75000.0
1,75000.0,75000.0
3,100000.0,110000.0
5,75000.0,130000.0
7,150000.0,205000.0
9,60000.0,120000.0
11,130000.0,130000.0
12,100000.0,165000.0
13,120000.0,165000.0
14,60096.0,87045.0


In [403]:
final_salary = salaries.median(axis=1)

In [404]:
final_salary = pd.DataFrame(final_salary)

In [412]:
final_salary = final_salary.rename(columns = {0:'final_salary'})

In [413]:
final_salary.head()

,final_salary
0,72500.0
1,75000.0
3,105000.0
5,102500.0
7,177500.0


In [414]:
jobs = pd.concat([job_details_2, final_salary], axis=1)

In [415]:
jobs = jobs.drop('salary', axis=1)

In [416]:
jobs

,location,title,company,final_salary
0,"Boston, MA",SAS Statistical Data Analyst-Healthcare Marketing,ALC Staffing Associates,72500.0
1,"Boston, MA",SAS Program / Statistical Analyst-Data Mining,Alexander Bec,75000.0
3,"Boston, MA 02108 (Back Bay-Beacon Hill area)",Data Analyst,Infotek Consulting Services Inc.,105000.0
5,"Cambridge, MA",Front-End Developer - JavaScript/HTML5 for EdT...,The Bivium Group,102500.0
7,"Houston, TX",Quantitative Risk Analyst,SearchBankingJobs,177500.0
9,"Lexington, MA 02420",Statistical Programmer,BioPier Inc.,90000.0
11,"Boston, MA 02116 (South End area)",Back End Java Developer-Machine Learning,Jobspring Partners,130000.0
12,"Boston, MA",Spark/Hadoop/Scala at Top Growth Stage Startup,Workbridge Associates,132500.0
13,"Boston, MA",Principal Software Engineer Lead Software Engi...,The Bivium Group,142500.0
14,"Boston, MA",Clinical Laboratory Surveyor,Commonwealth of Massachusetts,73570.5


In [417]:
jobs.dtypes

location         object
title            object
company          object
final_salary    float64
dtype: object

### Save your results as a CSV

In [418]:
# Export to csv
job_details_csv = jobs.to_csv

In [419]:
job_details_csv

<bound method DataFrame.to_csv of                                          location  \
0                                      Boston, MA   
1                                      Boston, MA   
3    Boston, MA 02108 (Back Bay-Beacon Hill area)   
5                                   Cambridge, MA   
7                                     Houston, TX   
9                             Lexington, MA 02420   
11              Boston, MA 02116 (South End area)   
12                                     Boston, MA   
13                                     Boston, MA   
14                                     Boston, MA   
15                                    Houston, TX   
16                               Quincy, MA 02169   
19      Cambridge, MA 02138 (West Cambridge area)   
23                                  Braintree, MA   
24                                 Scottsdale, AZ   
25                                    Chicago, IL   
26                                    Chicago, IL   
27          

## Predicting salaries using Random Forests + Another Classifier

#### Load in the the data of scraped salaries

In [616]:
## YOUR CODE HERE
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split

In [617]:
rfc = RandomForestClassifier()
knn = KNeighborsClassifier()

#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

We could also perform Linear Regression (or any regression) to predict the salary value here. Instead, we are going to convert this into a _binary_ classification problem, by predicting two classes, HIGH vs LOW salary.

While performing regression may be better, performing classification may help remove some of the noise of the extreme salaries. We don't have to choice the `median` as the splitting point - we could also split on the 75th percentile or any other reasonable breaking point.

In fact, the ideal scenario may be to predict many levels of salaries, 

In [424]:
# Median salary is 72,500 per year 
# Should do the mean or the 50th percentile instead as there aren't that many salaries above 72,500.

In [427]:
jobs.final_salary.describe()

count        34.000000
mean     112066.794118
std       40383.170629
min       35000.000000
25%       76250.000000
50%      108750.000000
75%      140000.000000
max      185000.000000
Name: final_salary, dtype: float64

In [428]:
#Upper 50% above 108,750
jobs['high_or_low'] = jobs['final_salary'].map(lambda x: 1 if x > 108750 else 0) 

In [429]:
jobs

,location,title,company,final_salary,high_or_low
0,"Boston, MA",SAS Statistical Data Analyst-Healthcare Marketing,ALC Staffing Associates,72500.0,0
1,"Boston, MA",SAS Program / Statistical Analyst-Data Mining,Alexander Bec,75000.0,0
3,"Boston, MA 02108 (Back Bay-Beacon Hill area)",Data Analyst,Infotek Consulting Services Inc.,105000.0,0
5,"Cambridge, MA",Front-End Developer - JavaScript/HTML5 for EdT...,The Bivium Group,102500.0,0
7,"Houston, TX",Quantitative Risk Analyst,SearchBankingJobs,177500.0,1
9,"Lexington, MA 02420",Statistical Programmer,BioPier Inc.,90000.0,0
11,"Boston, MA 02116 (South End area)",Back End Java Developer-Machine Learning,Jobspring Partners,130000.0,1
12,"Boston, MA",Spark/Hadoop/Scala at Top Growth Stage Startup,Workbridge Associates,132500.0,1
13,"Boston, MA",Principal Software Engineer Lead Software Engi...,The Bivium Group,142500.0,1
14,"Boston, MA",Clinical Laboratory Surveyor,Commonwealth of Massachusetts,73570.5,0


In [448]:
jobs_with_locations = pd.concat([jobs, pd.get_dummies(jobs.location)], axis=1)

In [450]:
jobs_with_locations.head(3)

,location,title,company,final_salary,high_or_low,"Atlanta, GA","Boston, MA","Boston, MA 02108 (Back Bay-Beacon Hill area)","Boston, MA 02116 (South End area)","Braintree, MA",...,"New York, NY 10013 (Tribeca area)","New York, NY 10167 (Midtown area)","Philadelphia, PA","Plano, TX","Quincy, MA 02169","San Fernando, CA","San Francisco, CA","Sausalito, CA 94965","Scottsdale, AZ","Seattle, WA"
0,"Boston, MA",SAS Statistical Data Analyst-Healthcare Marketing,ALC Staffing Associates,72500.0,0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Boston, MA",SAS Program / Statistical Analyst-Data Mining,Alexander Bec,75000.0,0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Boston, MA 02108 (Back Bay-Beacon Hill area)",Data Analyst,Infotek Consulting Services Inc.,105000.0,0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Thought experiment: What is the baseline accuracy for this model?

#### Create a Random Forest model to predict High/Low salary. Start by ONLY using the location as a feature. 

In [612]:
## YOUR CODE HERE
X_1 = jobs_with_locations.drop(jobs_with_locations[[0,1,2,3,4]], axis=1)
y_1 = jobs_with_locations.high_or_low

In [ ]:
# NO TRAIN TEST SPLIT. 

In [490]:
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [491]:
#Accuracy score.
rfc.score(X_train,y_train)

0.88235294117647056

In [532]:
from sklearn.cross_validation import cross_val_score

In [493]:
cross_val_score(rfc, X_train, y_train)

array([ 0.57142857,  0.4       ,  0.4       ])

In [ ]:
#The accuracy is high, but the cross validation score expresses substantially less 
# confidence. May be due to the smallness of the sample. 

In [614]:
#Try with KNN

In [668]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [669]:
knn.score(X_train,y_train)

0.58823529411764708

In [621]:
#Less accurate than with rfc.

In [622]:
cross_val_score(knn, X_train, y_train)

array([ 0.57142857,  0.6       ,  0.6       ])

In [ ]:
#Same cross val score as with rfc. 

#### Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' is in the title 
- or whether 'Manager' is in the title. 
- Then build a new Random Forest with these features. Do they add any value? 


In [520]:
## YOUR CODE HERE
senior_variable = jobs['title'].map(lambda x: 1 if 'Senior' in x else 0) 

In [521]:
senior_variable = pd.DataFrame(senior_variable)

In [523]:
senior_variable = senior_variable.rename(columns = {'title':'senior_variable'})

In [524]:
senior_variable

,senior_variable
0,0
1,0
3,0
5,0
7,0
9,0
11,0
12,0
13,0
14,0


In [525]:
jobs_with_seniors = pd.concat([jobs, senior_variable], axis=1)

In [526]:
jobs_with_seniors

,location,title,company,final_salary,high_or_low,senior_variable
0,"Boston, MA",SAS Statistical Data Analyst-Healthcare Marketing,ALC Staffing Associates,72500.0,0,0
1,"Boston, MA",SAS Program / Statistical Analyst-Data Mining,Alexander Bec,75000.0,0,0
3,"Boston, MA 02108 (Back Bay-Beacon Hill area)",Data Analyst,Infotek Consulting Services Inc.,105000.0,0,0
5,"Cambridge, MA",Front-End Developer - JavaScript/HTML5 for EdT...,The Bivium Group,102500.0,0,0
7,"Houston, TX",Quantitative Risk Analyst,SearchBankingJobs,177500.0,1,0
9,"Lexington, MA 02420",Statistical Programmer,BioPier Inc.,90000.0,0,0
11,"Boston, MA 02116 (South End area)",Back End Java Developer-Machine Learning,Jobspring Partners,130000.0,1,0
12,"Boston, MA",Spark/Hadoop/Scala at Top Growth Stage Startup,Workbridge Associates,132500.0,1,0
13,"Boston, MA",Principal Software Engineer Lead Software Engi...,The Bivium Group,142500.0,1,0
14,"Boston, MA",Clinical Laboratory Surveyor,Commonwealth of Massachusetts,73570.5,0,0


In [606]:
X_2 = jobs_with_seniors.drop(jobs_with_seniors[[0,1,2,3,4]], axis=1)
y_2 = jobs_with_seniors.high_or_low

In [528]:
X_2

,senior_variable
0,0
1,0
3,0
5,0
7,0
9,0
11,0
12,0
13,0
14,0


In [624]:
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [625]:
rfc.score(X_train,y_train)

0.6470588235294118

In [ ]:
#Not the most accurate for "senior" in job title, but could be because of the small size of sample

In [626]:
cross_val_score(rfc, X_train, y_train)

array([ 0.57142857,  0.6       ,  0.6       ])

In [ ]:
# "Senior" not highly predictive of high salaries, at least from this dataset

In [671]:
knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [672]:
knn.score(X_train,y_train)

0.58823529411764708

In [630]:
cross_val_score(knn, X_train, y_train)

array([ 0.57142857,  0.6       ,  0.6       ])

In [597]:
# Saw a few high salaries with "Quantitative" in there--should test for that 

In [598]:
quant_variable = jobs['title'].map(lambda x: 1 if 'Quantitative' in x else 0)  

In [600]:
quant_variable = pd.DataFrame(quant_variable)
quant_variable = quant_variable.rename(columns = {'title':'quant_variable'})

In [601]:
jobs_with_quant = pd.concat([jobs, quant_variable], axis=1) 

In [603]:
jobs_with_quant

,location,title,company,final_salary,high_or_low,quant_variable
0,"Boston, MA",SAS Statistical Data Analyst-Healthcare Marketing,ALC Staffing Associates,72500.0,0,0
1,"Boston, MA",SAS Program / Statistical Analyst-Data Mining,Alexander Bec,75000.0,0,0
3,"Boston, MA 02108 (Back Bay-Beacon Hill area)",Data Analyst,Infotek Consulting Services Inc.,105000.0,0,0
5,"Cambridge, MA",Front-End Developer - JavaScript/HTML5 for EdT...,The Bivium Group,102500.0,0,0
7,"Houston, TX",Quantitative Risk Analyst,SearchBankingJobs,177500.0,1,1
9,"Lexington, MA 02420",Statistical Programmer,BioPier Inc.,90000.0,0,0
11,"Boston, MA 02116 (South End area)",Back End Java Developer-Machine Learning,Jobspring Partners,130000.0,1,0
12,"Boston, MA",Spark/Hadoop/Scala at Top Growth Stage Startup,Workbridge Associates,132500.0,1,0
13,"Boston, MA",Principal Software Engineer Lead Software Engi...,The Bivium Group,142500.0,1,0
14,"Boston, MA",Clinical Laboratory Surveyor,Commonwealth of Massachusetts,73570.5,0,0


In [605]:
X_3 = jobs_with_quant.quant_variable
y_3 = jobs_with_quant.high_or_low

In [673]:
X_train, X_test, y_train, y_test = train_test_split(X_3, y_3, test_size=0.5, random_state=50)

In [674]:
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [609]:
rfc.score(X_train,y_train)

0.58823529411764708

In [610]:
#Accuracy score likely lower than usual because of the limited sample from the scraping.

In [611]:
cross_val_score(rfc, X_train,y_train)

array([ 0.42857143,  0.6       ,  0.4       ])

In [ ]:
# Not predictive according to cross val score. 

In [675]:
knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [676]:
knn.score(X_train,y_train)

0.58823529411764708

In [677]:
cross_val_score(knn,X_train,y_train)

array([ 0.57142857,  0.6       ,  0.6       ])

#### Rebuild this model with scikit-learn.
- You can either create the dummy features manually or use the `dmatrix` function from `patsy`
- Remember to scale the feature variables as well!


In [659]:
#I've already turned them to dummies, so I'm not sure scaling would be beneficial.

In [476]:
## YOUR CODE HERE

#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy of the model. 

In [ ]:
## YOUR CODE HERE

#### Random Forest Regressor

Let's try treating this as a regression problem. 

- Train a random forest regressor on the regression problem and predict your dependent.
- Evaluate the score with a 5-fold cross-validation
- Do a scatter plot of the predicted vs actual scores for each of the 5 folds, do they match?

In [635]:
## YOUR CODE HERE
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()

In [ ]:
#X_1, y_1: location; X_2, y_2: senior; X_3, y_3: quant

In [636]:
#First for location
X_train, X_test, y_train, y_test = train_test_split(X_1, y_1, test_size=0.5, random_state=50)

In [637]:
rfr.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [665]:
import matplotlib.pyplot as plt

In [638]:
rfr.score(X_train,y_train)

0.59722817460317446

In [639]:
#A slightly improved score from others.

In [642]:
cross_val_score(rfr, X_train, y_train, cv=5, scoring='mean_squared_error')

array([-0.18777778, -0.4975    , -0.47893333, -0.66      , -0.32      ])

In [ ]:
#All negative scores. 

In [645]:
#Now for "seniors" 

In [646]:
X_train, X_test, y_train, y_test = train_test_split(X_2, y_2, test_size=0.5, random_state=50)

In [647]:
rfr.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [648]:
rfr.score(X_train,y_train)

0.083613968253968163

In [649]:
#Not accurate at all.

In [650]:
cross_val_score(rfr, X_train, y_train, cv=5, scoring='mean_squared_error')

array([-0.1895362 , -0.26093463, -0.3225    , -0.3435363 , -0.21301775])

In [ ]:
#Again, negative scores.

In [651]:
#Maybe Quantitative could work.

In [653]:
X_train, X_test, y_train, y_test = train_test_split(X_3, y_3, test_size=0.5, random_state=50)

In [654]:
rfr.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [655]:
rfr.score(X_train, y_train)

0.086571800833340373

In [656]:
#Also very low. 

In [657]:
cross_val_score(rfr, X_train, y_train, cv=5, scoring='mean_squared_error')

array([-0.26600446, -0.25525259, -0.29889602, -0.25791682, -0.3144752 ])

#### Continue to incorporate other text features from the title or summary that you believe will predict the salary and examine their coefficients

In [ ]:
## YOUR CODE HERE

In [ ]:
## YOUR CODE HERE